In [ ]:
import time
import cv2
import HandTrackingModule as htm 
import os
import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
from keras import datasets, layers, models
from collections import Counter

In [ ]:
model1 = models.load_model("pre-trained-mnist/MNIST-digit-vgg.h5")
model2 = models.load_model("pre-trained-mnist/MNIST-numbers-cnn.h5")
model3 = models.load_model("pre-trained-mnist/mnist-cnn.h5")

In [ ]:
cap = cv2.VideoCapture(0)

w_cam, h_cam = 1280, 720
cap.set(3, w_cam)
cap.set(4, h_cam)

folderPath = "images/DrawingCanvas"
myList = os.listdir(folderPath)
# print(myList)

overlaylist = []

for imPath in myList:
  image = cv2.imread(f'{folderPath}/{imPath}')
  overlaylist.append(image)
  
# print(len(overlaylist))

header = overlaylist[0]
drawcolor = (0,0,255)
xp, yp = 0 , 0 
LineThickness = 40
EraserThickness = 50

imgCanvas = np.zeros((720,1280,3),np.uint8)

detector = htm.handDetector(detectionCon= 0.55)

while True:
  success, img = cap.read()
  
  img = cv2.flip(img,1)
  
  h , w, c = overlaylist[0].shape

  img[0:h, 0:w] = header
  
  img = detector.findHands(img)
  
  lmList = detector.findPosition(img)
  
  if len(lmList) != 0:
    
    x1,y1 = lmList[8][1], lmList[8][2]
    x2,y2 = lmList[12][1], lmList[12][2]
    
    fingers = detector.fingersUp()
    # print(fingers)
    
    if fingers[1]==1 and fingers[2]==0:
      # print("drawing mode")
      cv2.circle(img,(x1,y1),10,drawcolor,cv2.FILLED)
      
      if xp == 0 and yp == 0:
        xp, yp = x1, y1
    
      if drawcolor == (100,100,100): #eraser
        cv2.line(img,(xp,yp),(x1,y1),(0,0,0),EraserThickness)
        cv2.line(imgCanvas,(xp,yp),(x1,y1),(0,0,0),EraserThickness)
      
      else:
        cv2.line(img,(xp,yp),(x1,y1),drawcolor,LineThickness)
        cv2.line(imgCanvas,(xp,yp),(x1,y1),drawcolor,LineThickness)
      
      xp, yp =x1, y1
      
  
    if fingers[1]==1 and fingers[2]==1:
      # print("selection mode")
      xp, yp = 0 , 0 
      # cv2.circle(img,(x1,y1),5,(0,255,0),2,cv2.FILLED)
      # cv2.circle(img,(x2,y2),5,(0,255,0),2,cv2.FILLED)
      if y1 < h+30:
        
        if 0<x1<90:
          header = overlaylist[1]
          drawcolor = (0,0,255)
          
        elif 90<x1<180:
          header = overlaylist[2]
          drawcolor = (180,180,180)
          
        elif 180<x1<270:
          header = overlaylist[3]
          drawcolor = (0,255,0)
          
        elif 270<x1<360:
          header = overlaylist[4]
          drawcolor = (255,0,255)
          
        elif 360<x1<420:
          header = overlaylist[5]
          drawcolor = (255,0,0)
          
        elif 420<x1<550:
          header = overlaylist[6]
          drawcolor = (255,255,255)
          
        elif 1050<x1<1280:
          header = overlaylist[7]
          drawcolor = (100,100,100)
          
        elif 600<x1<1000:
          header = overlaylist[0]
          
          #########################################################################
          imgCanvas_resized = cv2.resize(imgCanvas, (28, 28))
          imgCanvas_grayscale = cv2.cvtColor(imgCanvas_resized, cv2.COLOR_BGR2GRAY)
          imgCanvas_reshaped = np.reshape(imgCanvas_grayscale, (28, 28))
          imgCanvas_normalized = imgCanvas_reshaped.astype('float32') / 255.0

          # plt.imshow(imgCanvas_normalized, cmap='gray')
          # plt.title('Preprocessed Image')
          # plt.show()

          imgCanvas_input = imgCanvas_normalized.reshape(-1, 28, 28, 1)

          prediction1 = model1.predict(imgCanvas_input)
          prediction2 = model2.predict(imgCanvas_input)
          prediction3 = model3.predict(imgCanvas_input)
          
          prediction1 = np.argmax(prediction1)
          prediction2 = np.argmax(prediction2)
          prediction3 = np.argmax(prediction3)
          
          prediction_votes = [prediction1, prediction2, prediction3]
          prediction = Counter(prediction_votes)
          most_confident_prediction, count = prediction.most_common(1)[0]

          predicted_digit = most_confident_prediction

          print(f"predicted_digit = {predicted_digit}")
          
          cv2.putText(img,f"{predicted_digit}",q
                      (100,400),cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),3)
      
          ###########################################################################
          
      cv2.rectangle(img,(x1,y1-25),(x2,y2+25),drawcolor,cv2.FILLED)
          
    if fingers.count(1)==4:
      imgCanvas = np.zeros((720,1280,3),np.uint8)
          
                  
  img = cv2.addWeighted(img,1,imgCanvas,1,0)
  
  
  # imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
  # _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
  # imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
  # img = cv2.bitwise_and(img,imgInv)
  # img = cv2.bitwise_or(img,imgCanvas)
  
  cv2.imshow("Drawing Canvas",img)
  # cv2.imshow("Canvas",imgCanvas)
  # cv2.imshow("Inv", imgInv)
  
  cv2.waitKey(1)
  if cv2.waitKey(1) & 0xFF == ord('q'): 
    break  
  
cap.release()
cv2.destroyAllWindows()